In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data',
                  sep = ",", header = None)
data.head()
data = pd.DataFrame.to_numpy(data)
X, y = data[:,:34].astype(np.float64), data[:,34]
le2 = LabelEncoder()
y = le2.fit_transform(y)
G = len(np.unique(y))
X = np.delete(X,[0,1], axis = 1)
for g in range(G):
  print(sum(y==g))
X.shape

126
225


(351, 32)

In [3]:
missing_rates = np.array([0.5, 0.6, 0.7, 0.8, 0.9])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([100,-80])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [4]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [5]:
leftindex = pd.MultiIndex.from_product([["50%", "60%", "70%", "80%", "90%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM          0.755$\pm$0.03  0.933$\pm$0.015   
                         LR           0.732$\pm$0.06  0.827$\pm$0.017   
                         DCT         0.681$\pm$0.053  0.822$\pm$0.041   
             sensitivity SVM          0.75$\pm$0.033  0.933$\pm$0.015   
                         LR          0.729$\pm$0.063  0.829$\pm$0.014   
                         DCT         0.675$\pm$0.054  0.821$\pm$0.041   
60%          F-1 score   SVM         0.703$\pm$0.026  0.912$\pm$0.032   
                         LR          0.742$\pm$0.075  0.808$\pm$0.047   
                         DCT         0.684$\pm$0.061  0.758$\pm$0.046   
             sensitivity SVM         0.698$\pm$0.025  0.911$\pm$0.034   
                         LR           0.738$\pm$0.08  0.809$\pm$0.048   
                         DCT         0.677$\pm$0.061   0.755$\pm$0.05   
70%          F-1 score   SVM         0.698$\pm$0.021  0.877$\pm$0.034   
                         LR          0.743$\pm$0.044  0.802$\pm$0.038   
                         DCT          0.69$\pm$0.079  0.706$\pm$0.063   
             sensitivity SVM         0.697$\pm$0.023  0.876$\pm$0.034   
                         LR          0.743$\pm$0.045  0.804$\pm$0.039   
                         DCT         0.689$\pm$0.076  0.711$\pm$0.067   
80%          F-1 score   SVM         0.681$\pm$0.039  0.828$\pm$0.051   
                         LR          0.699$\pm$0.047  0.785$\pm$0.035   
                         DCT          0.59$\pm$0.054  0.698$\pm$0.072   
             sensitivity SVM         0.685$\pm$0.034  0.825$\pm$0.052   
                         LR          0.695$\pm$0.049  0.787$\pm$0.036   
                         DCT         0.596$\pm$0.041  0.696$\pm$0.074   
90%          F-1 score   SVM         0.597$\pm$0.067  0.705$\pm$0.052   
                         LR          0.673$\pm$0.076  0.631$\pm$0.093   
                         DCT         0.525$\pm$0.147  0.591$\pm$0.095   
             sensitivity SVM          0.613$\pm$0.05  0.703$\pm$0.047   
                         LR          0.672$\pm$0.076  0.634$\pm$0.097   
                         DCT         0.549$\pm$0.113  0.589$\pm$0.103   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
50%          F-1 score   SVM          0.925$\pm$0.01  0.932$\pm$0.017   
                         LR          0.814$\pm$0.033  0.825$\pm$0.018   
                         DCT          0.82$\pm$0.056  0.825$\pm$0.037   
             sensitivity SVM          0.926$\pm$0.01  0.932$\pm$0.017   
                         LR          0.818$\pm$0.032  0.826$\pm$0.017   
                         DCT         0.821$\pm$0.055  0.823$\pm$0.038   
60%          F-1 score   SVM          0.896$\pm$0.04  0.912$\pm$0.025   
                         LR           0.81$\pm$0.039  0.801$\pm$0.046   
                         DCT         0.766$\pm$0.037  0.742$\pm$0.067   
             sensitivity SVM         0.895$\pm$0.042  0.911$\pm$0.026   
                         LR          0.811$\pm$0.041  0.801$\pm$0.047   
                         DCT         0.765$\pm$0.039   0.74$\pm$0.071   
70%          F-1 score   SVM         0.814$\pm$0.046   0.877$\pm$0.03   
                         LR           0.788$\pm$0.03  0.807$\pm$0.033   
                         DCT         0.753$\pm$0.031  0.734$\pm$0.037   
             sensitivity SVM         0.812$\pm$0.047  0.876$\pm$0.031   
                         LR           0.79$\pm$0.034  0.809$\pm$0.034   
                         DCT         0.757$\pm$0.035   0.734$\pm$0.04   
80%          F-1 score   SVM          0.71$\pm$0.031  0.831$\pm$0.047   
                         LR          0.692$\pm$0.053  0.784$\pm$0.031   
                         DCT         0.658$\pm$0.046  0.646$\pm$0.083   
             sensitivity SVM

In [6]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_Inosphere_high.csv", index = False)

In [7]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
50\% & F-1 score & SVM &   0.755\$\textbackslash pm\$0.03 &  0.933\$\textbackslash pm\$0.015 &   0.925\$\textbackslash pm\$0.01 &  0.932\$\textbackslash pm\$0.017 &  0.856\$\textbackslash pm\$0.045 &  0.933\$\textbackslash pm\$0.014 \\
    &             & LR &   0.732\$\textbackslash pm\$0.06 &  0.827\$\textbackslash pm\$0.017 &  0.814\$\textbackslash pm\$0.033 &  0.825\$\textbackslash pm\$0.018 &  0.813\$\textbackslash pm\$0.041 &  0.823\$\textbackslash pm\$0.026 \\
    &             & DCT &  0.681\$\textbackslash pm\$0.053 &  0.822\$\textbackslash pm\$0.041 &   0.82\$\textbackslash pm\$0.056 &  0.825\$\textbackslash pm\$0.037 &   0.74\$\textbackslash p